# Анализ сервиса для чтения книг по подписке

__Цель проекта:__

Сформулировать ценностное предложение для нового продукта - сервиса для чтения книг по подписке.

__Задачи проекта:__

1. Провести анализ базы данных сервиса для чтения книг по подписке
2. Сформировать выводы и предложения по результатам анализа.

## Получение и анализ данных

In [1]:
# импортируем библиотеки и получаем данные
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Исследуем таблицы базы данных: books, authors, publishers, ratings, reviews

In [2]:
# список таблиц базы данных
sheets = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for i in sheets:
    query1 = 'select * from  {}'.format(i)
    print()
    df = pd.io.sql.read_sql(query1, con = engine)
    print('Таблица:', i, 'Количество записей = ', len(df))
    display(df.head())


Таблица: books Количество записей =  1000


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Таблица: authors Количество записей =  636


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Таблица: publishers Количество записей =  340


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Таблица: ratings Количество записей =  6456


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Таблица: reviews Количество записей =  2793


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Выводы: данные таблиц загружаются корректно.

## Оценка количества книг вышедших после 1 января 2000 года

In [3]:
query2 = '''
SELECT COUNT(book_id) 
FROM  books
WHERE CAST(publication_date AS date) > '2000-01-01'

'''
pd.io.sql.read_sql(query2, con = engine)

,count
0,819


Вывод: В сервисе представлено 819 книг выпушенных после 1 января 2000 года.

## Определение для каждой книги количества обзоров и средней оценки

In [4]:
query3 = '''
SELECT b.title,
    COUNT(DISTINCT rv.review_id) AS count_reviews,
    AVG(rt.rating) AS avg_rating
FROM  books as b
LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
ORDER BY count_reviews, avg_rating

'''
pd.io.sql.read_sql(query3, con = engine)

,title,count_reviews,avg_rating
0,The Natural Way to Draw,0,3.000000
1,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
2,Leonardo's Notebooks,0,4.000000
3,Essential Tales and Poems,0,4.000000
4,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
...,...,...,...
995,The Glass Castle,6,4.206897
996,The Book Thief,6,4.264151
997,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
998,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634


Вывод: Получены данные о количествt обзоров и средней оценке книг. Не у всех книг есть обзоры.

## Определение издательства, которое выпустило наибольшее число книг

In [5]:
query4 = '''
WITH
i AS (SELECT * -- выделение книг как изданий числом страниц более 50
    FROM  books as b
    INNER JOIN publishers AS pb ON b.publisher_id = pb.publisher_id
    WHERE CAST(b.num_pages AS int) > 50)

SELECT publisher,
    COUNT(publisher) AS count_rating
FROM i
GROUP BY publisher
ORDER BY count_rating DESC
LIMIT 1;

'''
pd.io.sql.read_sql(query4, con = engine)

,publisher,count_rating
0,Penguin Books,42


Вывод: Опредлено издательство выпустившее максимальное количество книг находящихся в сервисе - Penguin Books.

## Определение автора с самой высокой средней оценкой книг

In [6]:
query5 = '''
WITH
i AS (SELECT book_id -- список id книг с числом оценок более 50
    FROM  ratings
    GROUP BY book_id
    HAVING COUNT(rating) > 50)

SELECT
    a.author,
    AVG(r.rating)
FROM i
LEFT JOIN books AS b ON b.book_id=i.book_id
INNER JOIN authors AS a ON a.author_id=b.author_id
INNER JOIN ratings AS r ON r.book_id=b.book_id
GROUP BY a.author
ORDER BY AVG(r.rating) DESC
LIMIT 1;

'''
pd.io.sql.read_sql(query5, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Вывод: определен автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

## Определение среднего количества обзоров от пользователей, которые поставили больше 50 оценок

In [7]:
query7 = '''
WITH
i AS (SELECT -- выделение пользователей оставивших более 50 обзоров
         username,
         COUNT(username) AS count_rating
     FROM ratings
     GROUP BY username
     HAVING COUNT(username) > 50)

SELECT
    AVG(count) AS avg_review
FROM
(SELECT rev.username, COUNT(rev.review_id)
FROM i
INNER JOIN reviews AS rev ON rev.username = i.username
GROUP BY rev.username) AS ii

'''
pd.io.sql.read_sql(query7, con = engine)

,avg_review
0,24.333333


Вывод: определено среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.3

## Выводы и предложения по результатам анализа

По результатам анализа получены ответы на поставленные в задании вопросы:
- оценить количество книг вышедших после 1 января 2000 года: 819 штук (82%)
- определены для каждой книги количество обзоров и средние оценки, обзоры есть не у каждой книги
- определить издательство, которое выпустило наибольшее число книг: Penguin Books
- определить автора с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré
- определить среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.3

Предложения:
- разработать программу рейтингования пользователей оставляющих обзоры, средний балл установить на 24 обзора
- разработать партнерскую программу с издательством Penguin Books
- проработать маркетинговую акцию по продвижению книг J.K. Rowling/Mary GrandPré среди пользователей
- проработать привлечение целевой аудитории читателей современных книг